In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import json
import os
from datetime import datetime

FILE_NAME = "tasks.json"

class ToDoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("To-Do List App")
        self.tasks = []

        # UI Elements
        self.task_entry = tk.Entry(root, width=40)
        self.task_entry.grid(row=0, column=0, padx=10, pady=10, columnspan=2)

        self.add_button = tk.Button(root, text="Add Task", command=self.add_task)
        self.add_button.grid(row=0, column=2, padx=5)

        self.task_listbox = tk.Listbox(root, width=60, height=15)
        self.task_listbox.grid(row=1, column=0, columnspan=3, padx=10)

        self.edit_button = tk.Button(root, text="Edit Task", command=self.edit_task)
        self.edit_button.grid(row=2, column=0, pady=5)

        self.delete_button = tk.Button(root, text="Delete Task", command=self.delete_task)
        self.delete_button.grid(row=2, column=1, pady=5)

        self.done_button = tk.Button(root, text="Mark as Done", command=self.mark_done)
        self.done_button.grid(row=2, column=2, pady=5)

        self.save_button = tk.Button(root, text="Save Tasks", command=self.save_tasks)
        self.save_button.grid(row=3, column=0, pady=10)

        self.load_button = tk.Button(root, text="Load Tasks", command=self.load_tasks)
        self.load_button.grid(row=3, column=2, pady=10)

        # Load tasks from file if available
        self.load_tasks()

    def add_task(self):
        task_text = self.task_entry.get().strip()
        if task_text == "":
            messagebox.showwarning("Input Error", "Task cannot be empty.")
            return

        priority = simpledialog.askstring("Priority", "Enter priority (High/Medium/Low):")
        due_date = simpledialog.askstring("Due Date", "Enter due date (YYYY-MM-DD):")

        task = {
            "text": task_text,
            "priority": priority or "Medium",
            "due_date": due_date or "",
            "done": False
        }
        self.tasks.append(task)
        self.refresh_listbox()
        self.task_entry.delete(0, tk.END)

    def edit_task(self):
        selected = self.task_listbox.curselection()
        if not selected:
            messagebox.showwarning("Selection Error", "No task selected.")
            return
        index = selected[0]
        task = self.tasks[index]
        new_text = simpledialog.askstring("Edit Task", "Edit task text:", initialvalue=task["text"])
        if new_text:
            self.tasks[index]["text"] = new_text
            self.refresh_listbox()

    def delete_task(self):
        selected = self.task_listbox.curselection()
        if not selected:
            messagebox.showwarning("Selection Error", "No task selected.")
            return
        index = selected[0]
        del self.tasks[index]
        self.refresh_listbox()

    def mark_done(self):
        selected = self.task_listbox.curselection()
        if not selected:
            messagebox.showwarning("Selection Error", "No task selected.")
            return
        index = selected[0]
        self.tasks[index]["done"] = True
        self.refresh_listbox()

    def refresh_listbox(self):
        self.task_listbox.delete(0, tk.END)
        for task in self.tasks:
            status = "[Done] " if task["done"] else ""
            display = f"{status}{task['text']} (Priority: {task['priority']}, Due: {task['due_date']})"
            self.task_listbox.insert(tk.END, display)

    def save_tasks(self):
        with open(FILE_NAME, 'w') as f:
            json.dump(self.tasks, f, indent=4)
        messagebox.showinfo("Success", "Tasks saved successfully.")

    def load_tasks(self):
        if os.path.exists(FILE_NAME):
            with open(FILE_NAME, 'r') as f:
                self.tasks = json.load(f)
            self.refresh_listbox()


if __name__ == "__main__":
    root = tk.Tk()
    app = ToDoApp(root)
    root.mainloop()
